In [1]:
!python setup.py build_ext --inplace

Compiling games/breakthrough.py because it changed.
[1/1] Cythonizing games/breakthrough.py
running build_ext
building 'games.breakthrough' extension
clang -Wsign-compare -Wunreachable-code -fno-common -dynamic -DNDEBUG -g -fwrapv -O3 -Wall -arch arm64 -arch x86_64 -g -I/Users/tom/github/mcts_python/.venv/lib/python3.11/site-packages/numpy/core/include -I/Users/tom/github/mcts_python/.venv/include -I/Library/Frameworks/Python.framework/Versions/3.11/include/python3.11 -c games/breakthrough.c -o build/temp.macosx-10.9-universal2-cpython-311/games/breakthrough.o
In file included from games/breakthrough.c:1099:
In file included from /Users/tom/github/mcts_python/.venv/lib/python3.11/site-packages/numpy/core/include/numpy/arrayobject.h:5:
In file included from /Users/tom/github/mcts_python/.venv/lib/python3.11/site-packages/numpy/core/include/numpy/ndarrayobject.h:12:
In file included from /Users/tom/github/mcts_python/.venv/lib/python3.11/site-packages/numpy/core/include/numpy/ndarraytype

In [2]:
import collections
from games.gamestate import GameState
from random import choices
import time
from scipy.stats import chisquare

def test_random_action_distribution(state: GameState):
    num_trials = 10000
    generated_actions = [state.get_random_action() for _ in range(num_trials)]

    all_possible_actions = state.get_legal_actions()
    expected_distribution = [1 / len(all_possible_actions)] * len(all_possible_actions)  # uniform distribution

    # count how often each action was generated
    observed_distribution = [generated_actions.count(action) / num_trials for action in all_possible_actions]

    # Perform chi-square test
    chisq, p = chisquare(f_obs=observed_distribution, f_exp=expected_distribution)

    # Here you can use a p-value threshold of 0.05 (common in statistics), but depending on your specific requirements, you may choose a different threshold.
    assert p > 0.05, "The distribution of the generated actions differs from the expected distribution, p-value: {}".format(p)


# TODO Hier was je gebleven. Even checken of dit allemaal klopt voor je met MCTS verder gaat.
def test_random_action_coverage(state: GameState):
    num_trials = 20000
    generated_actions = [state.get_random_action() for _ in range(num_trials)]

    unique_generated_actions = set(generated_actions)
    all_possible_actions = set(state.get_legal_actions())
    
    # Here, we check if all possible actions were generated.
    assert unique_generated_actions == all_possible_actions, f"Random actions: {len(unique_generated_actions)}, all actions: {len(all_possible_actions)}, generated percentage: {len(unique_generated_actions) / len(all_possible_actions) * 100:.2f}%"

def test_random_action_uniformity(state: GameState):
    num_trials = 100000
    generated_actions = [state.get_random_action() for _ in range(num_trials)]
    
    action_counts = collections.Counter(generated_actions)
    min_count = min(action_counts.values())
    max_count = max(action_counts.values())
    
    # Here, we check if the actions are uniformly distributed. 
    assert max_count - min_count < num_trials * 0.05, f"Random actions are not uniformly distributed, max count: {max_count}, min count: {min_count}, num trials: {num_trials}"


def test_legal_actions_equality(game: GameState):
    actions_from_get = game.get_legal_actions()
    actions_from_yield = list(game.yield_legal_actions())

    # Check if the two sets of actions are equal
    are_equal = set(actions_from_get) == set(actions_from_yield)

    assert are_equal, "The generated moves from get_legal_actions and yield_legal_actions are not equal"

def test_get_legal_actions_uniqueness(game: GameState):
    actions_from_get = game.get_legal_actions()

    # Check if the generated moves from get function are unique
    is_unique_get = len(actions_from_get) == len(set(actions_from_get))

    assert is_unique_get, "The generated moves from get_legal_actions function are not unique"

def test_yield_legal_actions_uniqueness(game: GameState):
    actions_from_yield = list(game.yield_legal_actions())

    # Check if the generated moves from yield function are unique
    is_unique_yield = len(actions_from_yield) == len(set(actions_from_yield))

    assert is_unique_yield, "The generated moves from yield_legal_actions function are not unique"

def test_move_weights(state: GameState):
    # Get legal actions
    actions = state.get_legal_actions()

    # Get move weights
    weights = state.move_weights(actions)

    # Test that move_weights returns a list of weights 
    assert isinstance(weights, list), "move_weights should return a list,now: {}".format(type(weights))

    # Test that length of weights is same as actions
    assert len(weights) == len(actions), "Length of weights should be same as number of actions, now: {}".format(len(weights))

    # Test that all weights are >= 0
    for weight in weights:
        assert weight >= 0, "Weights should be greater than or equal to 0 (now: {})".format(weight)

    # Test that the sum of the weights is > 0 (assuming you have some actions to take)
    assert sum(weights) > 0, "The sum of weights should be greater than 0, now: {}".format(sum(weights))

def test_efficiency_get_vs_yield(game_class, num_trials=10, n_moves=20, return_time=False):
    # Prepare a list of states
    states = []
    game_class = type(game_class)
    # Create states with random moves
    for _ in range(num_trials):
        state = game_class()

        # Place random moves on the board
        for _ in range(n_moves):
            if state.is_terminal():
                break

            actions = state.get_legal_actions()
            test_move_weights(state)
            action = choices(actions, weights=state.move_weights(actions), k=1)[0]
            state = state.apply_action(action)

        states.append(state)

    # Measure time for get_legal_actions
    start_time_get = time.perf_counter()
    for _ in range(num_trials):
        for state in states:
            _ = state.get_legal_actions()
    end_time_get = time.perf_counter()
    total_time_get = end_time_get - start_time_get

    # Measure time for yield_legal_actions
    start_time_yield = time.perf_counter()
    for _ in range(num_trials):
        for state in states:
            _ = list(state.yield_legal_actions())
    end_time_yield = time.perf_counter()
    total_time_yield = end_time_yield - start_time_yield

    # print(f'Average time per action (get_legal_actions): {total_time_get / num_trials:.6f} seconds')
    # print(f'Average time per action (yield_legal_actions): {total_time_yield / num_trials:.6f} seconds')
    # print(f"get_legal_actions is {total_time_yield / total_time_get:.2f} times faster than yield_legal_actions")
    
    if return_time:
        return total_time_get, total_time_yield

In [3]:
from run_games import game_dict
from termcolor import colored
import random

failed_tests = {game: [] for game in game_dict.keys()}
failed_tests.pop("tictactoe") # This game is in the list twice, so skip it

tests = [
    ("Test Move Weights", test_move_weights),
    ("test_random_action_distribution", test_random_action_distribution),
    ("Random Action Coverage", test_random_action_coverage),
    ("Random Action Uniformity", test_random_action_uniformity),
    ("Legal Actions Equality", test_legal_actions_equality),
    ("Legal Get Actions Uniqueness", test_get_legal_actions_uniqueness),
    ("Legal Yield Actions Uniqueness", test_yield_legal_actions_uniqueness),
    ("Test Efficiency of Generator", test_efficiency_get_vs_yield),
]
n_tests_per_game = 10
# For each game, create an instance of the game and run the tests.
for game_name, game_class in game_dict.items():
    if game_name == "tictactoe":
        continue
    
    for i in range(n_tests_per_game):
        state = None
        while True:
            if game_name == "ninarow":
                state = game_class(board_size=9, row_length=5)
            else:
                state = game_class()

            # Place random moves on the board
            for n in range(1+random.randint(10, 30)):
                if state.is_terminal():
                    print(f"{game_name} Reached terminal state in {n} moves, trying again...")
                    break

                actions = state.get_legal_actions()
                try:
                    test_move_weights(state)
                except AssertionError as ex:
                    print(colored(f"{game_name} test 'test_move_weights' failed. {str(ex)}", 'red'))
                    print(state.visualize())
                    failed_tests[game_name].append("test_move_weights")
                    break
                
                action = choices(actions, weights=state.move_weights(actions), k=1)[0]
                state = state.apply_action(action)
            
            if not state.is_terminal():
                # Reached a non-terminal state, so we can run the tests
                break
            
        for test_name, test_func in tests:
            try:
                test_func(state)
                # print(colored(f"Test '{test_name}' passed.", 'green'))
            except AssertionError as ex:
                print(colored(f"{game_name} test '{test_name}' failed. {str(ex)}", 'red'))
                print(state.visualize())
                failed_tests[game_name].append(test_name)

# Print a summary of the failed tests at the end.
print("\n\n==== Summary of Failed Tests ====")
for game_name, failed_test_list in failed_tests.items():
    if failed_test_list:
        print(colored(f"Game: {game_name}", 'red'))
        for test_name in failed_test_list:
            print(colored(f"   Failed test: {test_name}", 'red'))
    else:
        print(colored(f"Game: {game_name} - All tests passed.", 'green'))

Transpositions is compiled.
Amazons is compiled.
Blokus is compiled.
Breakthrough is compiled.
Kalah is compiled.
Tictactoe is compiled.
breakthrough test 'Random Action Uniformity' failed. Random actions are not uniformly distributed, max count: 13145, min count: 2133, num trials: 100000
1 . B B B . B B B
2 B B . . B B . .
3 . . . B . . . .
4 . B W . . . B .
5 . . . . . . W .
6 W W . . . W . .
7 W . W W W . . W
8 W . . W W . W W
  A B C D E F G H
Player: 1



TypeError: Expected list, got numpy.ndarray

In [ ]:
from games.tictactoe import TicTacToeGameState
from numpy.random import choice as npchoice
import random
from ai.c_random import c_weighted_choice

state = TicTacToeGameState(board_size=9, row_length=5)
# Place random moves on the board
for n in range(1 + random.randint(10, 30)):
    if state.is_terminal():
        print(f"Reached terminal state in {n} moves, trying again...")
        break
    actions = state.get_legal_actions()
    action = random.choices(actions, weights=state.move_weights(actions), k=1)[0]
    state = state.apply_action(action)
print(state.visualize())

actions = state.get_legal_actions()
weights = state.move_weights(actions)

In [ ]:
%load_ext Cython

In [ ]:
%%cython
from games.tictactoe import TicTacToeGameState
from numpy.random import choice as npchoice
import random
from ai.c_random import c_weighted_choice
import time

state = TicTacToeGameState(board_size=9, row_length=5)
# Place random moves on the board
for n in range(1 + random.randint(10, 30)):
    if state.is_terminal():
        print(f"Reached terminal state in {n} moves, trying again...")
        break
    actions = state.get_legal_actions()
    action = random.choices(actions, weights=state.move_weights(actions), k=1)[0]
    state = state.apply_action(action)
print(state.visualize())

actions = state.get_legal_actions()
weights = state.move_weights(actions)
start_t = time.time()
for i in range(1000000):
    random.choices(actions, weights=weights, k=1)[0]
print(f"Took {time.time() - start_t} seconds to run 1,000,000 times")

start_t = time.time()
for i in range(1000000):
    actions[c_weighted_choice(weights)]
print(f"Took {time.time() - start_t} seconds to run 1,000,000 times")

In [ ]:
%load_ext Cython

In [ ]:
%%cython
import time
import random
from ai.c_random import c_shuffle
data = [(i, 1, 2)  for i in range(100)]


start_time = time.time()
for i in range(10**6):
    c_shuffle(data)
c_time = time.time() - start_time

start_time = time.time()
for i in range(10**6):
    random.shuffle(data)
p_time = time.time() - start_time

print(f"C time: {c_time}")
print(f"Python time: {p_time}")